# ETL: Raw → Silver Layer

## Contexto do Projeto

Este notebook realiza o processo completo de ETL (Extract, Transform, Load) dos dados brutos de Dengue (SINAN 2024-2025) para a camada Silver do Data Lake.

### Persona: Gestor de Vigilância Epidemiológica

**Objetivo:** Monitorar a evolução da dengue no Brasil, identificar regiões críticas e avaliar a efetividade das ações de controle.

**Questões de Negócio:**
- Quais UFs apresentam maior incidência de casos?
- Qual o perfil demográfico dos pacientes mais afetados?
- Qual a taxa de letalidade por região?
- Em qual período do ano ocorrem os picos epidêmicos?

---

## Processo ETL

1. **Extract**: Carrega dados brutos do arquivo CSV (SINAN)
2. **Transform**: Aplica limpeza, decodificação de campos SINAN e criação de métricas derivadas
3. **Load**: Carrega dados tratados no PostgreSQL (camada Silver)

## 1. Configuração e Imports

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_batch
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 80)

In [2]:
# =============================================================================
# CONFIGURAÇÕES DO PROJETO
# =============================================================================

# Caminhos dos arquivos
INPUT_FILE = '../Data_Layer/raw/DENGBR25.csv'
OUTPUT_DIR = '../Data_Layer/silver'
OUTPUT_CSV = 'dengue_silver.csv'

# Configuração do banco de dados PostgreSQL (Docker)
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'gis'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres')
}

print('Configuração definida:')
print(f'   Input: {INPUT_FILE}')
print(f'   Output: {OUTPUT_DIR}/{OUTPUT_CSV}')
print(f"   Banco: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

Configuração definida:
   Input: ../Data_Layer/raw/DENGBR25.csv
   Output: ../Data_Layer/silver/dengue_silver.csv
   Banco: localhost:5432/gis


## 2. Definição de Colunas

### Colunas Removidas e Justificativas

Com base na análise exploratória (bronze_analysis) e na persona do **Gestor de Vigilância Epidemiológica**, as seguintes colunas foram **removidas** por não agregarem valor às análises epidemiológicas:

| Critério | Colunas | Motivo da Remoção |
|----------|---------|-------------------|
| >90% nulos | Múltiplas colunas clínicas | Dados insuficientes para análise |
| Redundância | Códigos originais após mapeamento | Substituídos por descrições legíveis |
| Granularidade | Sintomas/Alarmes individuais | Agregados em contadores (qtd_sintomas, qtd_alarmes) |
| Irrelevância | Campos administrativos | Não contribuem para análise epidemiológica |

### Colunas Mantidas (Esquema Silver)

| Categoria | Colunas | Uso |
|-----------|---------|-----|
| 📍 Localização | uf_sigla | Análise geográfica |
| 📅 Tempo | data_notificacao, data_sintomas, semana_epi | Séries temporais |
| 👤 Demografia | idade_anos, faixa_etaria, sexo_desc, raca_desc | Perfil epidemiológico |
| 🏥 Clínico | qtd_sintomas, qtd_alarmes, fl_comorbidade, resultado_sorologia_desc, resultado_ns1_desc | Gravidade clínica e exames |
| 📋 Desfecho | classificacao_desc, evolucao_desc | Resultado do caso |
| 🚩 Flags | fl_confirmado, fl_grave, fl_obito, fl_hospitalizado | Indicadores |

In [3]:
# =============================================================================
# DEFINIÇÃO DE COLUNAS
# =============================================================================

# Colunas a serem REMOVIDAS (irrelevantes para análise epidemiológica)
COLUNAS_REMOVIDAS = [
    # >90% nulos (identificadas na análise bronze)
    'NDUPam_N', 'COMPLAM_N', 'NDUPAM_N', 'COUAM_AN',
    
    # Campos administrativos
    'NU_NOTIFIC', 'ID_AGRAVO', 'TP_NOT', 'ID_REGIONA', 'ID_UNIDADE',
    
    # Redundantes após mapeamento
    'SG_UF_NOT', 'CS_SEXO', 'CS_RACA', 'CS_ESCOL_N', 'CS_GESTANT',
    'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO', 'TPAUTOCTO', 'NU_IDADE_N',
    
    # Sintomas individuais (agregados em qtd_sintomas)
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO',
    'NAUSEA', 'DOR_COSTAS', 'ARTRALGIA', 'DOR_RETRO',
    
    # Alarmes individuais (agregados em qtd_alarmes)
    'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG',
    'ALRM_HEMAT', 'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT',
    
    # Comorbidades individuais (agregados em fl_comorbidade)
    'DIABETES', 'HIPERTENSA', 'RENAL', 'HEPATOPAT'
]

# Colunas MANTIDAS para processamento inicial
COLUNAS_PROCESSAMENTO = [
    # Localização
    'SG_UF_NOT',
    
    # Datas
    'DT_NOTIFIC', 'DT_SIN_PRI', 'DT_OBITO',
    
    # Dados do Paciente
    'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N',
    
    # Sintomas (para agregar)
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO',
    'NAUSEA', 'DOR_COSTAS', 'ARTRALGIA', 'DOR_RETRO',
    
    # Comorbidades (para agregar)
    'DIABETES', 'HIPERTENSA', 'RENAL', 'HEPATOPAT',
    
    # Exames laboratoriais
    'RESUL_SORO', 'RESUL_NS1',
    
    # Classificação e Evolução
    'HOSPITALIZ', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO',
    
    # Sinais de Alarme (para agregar)
    'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG',
    'ALRM_HEMAT', 'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT',
    
    # Autoctonia
    'TPAUTOCTO'
]

print(f'Colunas para processamento: {len(COLUNAS_PROCESSAMENTO)}')

Colunas para processamento: 37


## 3. Dicionários de Mapeamento (SINAN)

Os códigos do SINAN precisam ser convertidos para descrições legíveis.
Estes mapeamentos são baseados no Dicionário de Dados oficial do SINAN.

In [4]:
# =============================================================================
# DICIONÁRIOS DE MAPEAMENTO (SINAN)
# =============================================================================

UF_MAP = {
    11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO',
    21: 'MA', 22: 'PI', 23: 'CE', 24: 'RN', 25: 'PB', 26: 'PE', 27: 'AL',
    28: 'SE', 29: 'BA', 31: 'MG', 32: 'ES', 33: 'RJ', 35: 'SP', 41: 'PR',
    42: 'SC', 43: 'RS', 50: 'MS', 51: 'MT', 52: 'GO', 53: 'DF'
}

SEXO_MAP = {'M': 'Masculino', 'F': 'Feminino', 'I': 'Ignorado'}

RACA_MAP = {1: 'Branca', 2: 'Preta', 3: 'Amarela', 4: 'Parda', 5: 'Indigena', 9: 'Ignorado'}

ESCOLARIDADE_MAP = {
    0: 'Analfabeto', 1: '1-4 serie incompleta', 2: '4 serie completa',
    3: '5-8 serie incompleta', 4: 'Fundamental completo', 5: 'Medio incompleto',
    6: 'Medio completo', 7: 'Superior incompleto', 8: 'Superior completo',
    9: 'Ignorado', 10: 'Nao se aplica'
}

CLASSIFICACAO_MAP = {
    5: 'Descartado', 8: 'Inconclusivo', 10: 'Dengue',
    11: 'Dengue com Sinais de Alarme', 12: 'Dengue Grave', 13: 'Chikungunya'
}

CRITERIO_MAP = {1: 'Laboratorial', 2: 'Clinico-epidemiologico', 3: 'Em investigacao'}

EVOLUCAO_MAP = {
    1: 'Cura', 2: 'Obito pelo agravo', 3: 'Obito por outras causas',
    4: 'Obito em investigacao', 9: 'Ignorado'
}

GESTANTE_MAP = {
    1: '1 Trimestre', 2: '2 Trimestre', 3: '3 Trimestre',
    4: 'Idade gestacional ignorada', 5: 'Nao', 6: 'Nao se aplica', 9: 'Ignorado'
}

AUTOCTONE_MAP = {1: 'Autoctone', 2: 'Importado', 3: 'Indeterminado'}

# Resultados de exames laboratoriais
RESUL_SORO_MAP = {
    1: 'Reagente', 2: 'Nao Reagente', 3: 'Inconclusivo', 4: 'Nao Realizado', 9: 'Ignorado'
}

RESUL_NS1_MAP = {
    1: 'Positivo', 2: 'Negativo', 3: 'Inconclusivo', 4: 'Nao Realizado', 9: 'Ignorado'
}

print('Dicionários de mapeamento definidos:')

Dicionários de mapeamento definidos:


## 4. Extract - Carregamento dos Dados Brutos

In [5]:
# =============================================================================
# EXTRACT - CARREGAMENTO DOS DADOS
# =============================================================================

print('=' * 70)
print('ETAPA 1: EXTRACT')
print('=' * 70)

print(f'\nCarregando dados de: {INPUT_FILE}')

# Verificação do arquivo de entrada
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f'Arquivo não encontrado: {INPUT_FILE}')

try:
    df_raw = pd.read_csv(INPUT_FILE, encoding='latin-1', low_memory=False)
except Exception as e:
    raise Exception(f'Erro ao ler arquivo CSV: {e}')

print(f'\nDados carregados com sucesso!')
print(f'   Registros: {len(df_raw):,}')
print(f'   Colunas: {len(df_raw.columns)}')
print(f'   Memória: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB')
print(f'\nColunas disponíveis:')
print(list(df_raw.columns))

ETAPA 1: EXTRACT

Carregando dados de: ../Data_Layer/raw/DENGBR25.csv

Dados carregados com sucesso!
   Registros: 1,668,787
   Colunas: 121
   Memória: 2574.12 MB

Colunas disponíveis:
['TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI', 'ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS', 'DT_INVEST', 'ID_OCUPA_N', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT', 'AUTO_IMUNE', 'DT_CHIK_S1', 'DT_CHIK_S2', 'DT_PRNT', 'RES_CHIKS1', 'RES_CHIKS2', 'RESUL_PRNT', 'DT_SORO', 'RESUL_SORO', 'DT_NS1', 'RESUL_NS1', 'DT_VIRAL', 'RESUL_VI_N', 'DT_PCR', 'RESUL_PCR_', 'SOROTIPO', 'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'DT_INTERNA', 'UF', 'MUNICIPIO', 'TPAUTO

In [6]:
# Pré-visualização dos dados brutos
print('\nPrimeiras 5 linhas:')
df_raw.head()


Primeiras 5 linhas:


,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,SEM_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,SG_UF,ID_MN_RESI,ID_RG_RESI,ID_PAIS,DT_INVEST,ID_OCUPA_N,FEBRE,MIALGIA,CEFALEIA,EXANTEMA,VOMITO,NAUSEA,DOR_COSTAS,CONJUNTVIT,ARTRITE,ARTRALGIA,PETEQUIA_N,LEUCOPENIA,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,DT_CHIK_S1,DT_CHIK_S2,DT_PRNT,RES_CHIKS1,RES_CHIKS2,RESUL_PRNT,DT_SORO,RESUL_SORO,DT_NS1,RESUL_NS1,DT_VIRAL,RESUL_VI_N,DT_PCR,RESUL_PCR_,SOROTIPO,HISTOPA_N,IMUNOH_N,HOSPITALIZ,DT_INTERNA,UF,MUNICIPIO,TPAUTOCTO,COUFINF,COPAISINF,COMUNINF,CLASSI_FIN,CRITERIO,DOENCA_TRA,CLINC_CHIK,EVOLUCAO,DT_OBITO,DT_ENCERRA,ALRM_HIPOT,ALRM_PLAQ,ALRM_VOM,ALRM_SANG,ALRM_HEMAT,ALRM_ABDOM,ALRM_LETAR,ALRM_HEPAT,ALRM_LIQ,DT_ALRM,GRAV_PULSO,GRAV_CONV,GRAV_ENCH,GRAV_INSUF,GRAV_TAQUI,GRAV_EXTRE,GRAV_HIPOT,GRAV_HEMAT,GRAV_MELEN,GRAV_METRO,GRAV_SANG,GRAV_AST,GRAV_MIOC,GRAV_CONSC,GRAV_ORGAO,DT_GRAV,MANI_HEMOR,EPISTAXE,GENGIVO,METRO,PETEQUIAS,HEMATURA,SANGRAM,LACO_N,PLASMATICO,EVIDENCIA,PLAQ_MENOR,CON_FHD,COMPLICA,TP_SISTEMA,NDUPLIC_N,DT_DIGITA,CS_FLXRET,FLXRECEBI,MIGRADO_W
0,2,A90,2024-12-29,202501,2024,32,320070,32004.0,2485397.0,2024-12-29,202501,2003.0,4021,M,6.0,4.0,6.0,32,320070.0,32004.0,1,2024-12-29,10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,2.0,NaN,NaN,NaN,NaN,2025-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-29,NaN,NaN,NaN
1,2,A90,2024-12-29,202501,2024,32,320020,32004.0,2448025.0,2024-12-29,202501,1952.0,4072,F,5.0,1.0,1.0,32,320020.0,32004.0,1,2024-12-29,999990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,2.0,NaN,NaN,1.0,NaN,2025-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-29,NaN,NaN,NaN
2,2,A90,2024-12-29,202501,2024,32,320020,32004.0,2448025.0,2024-12-29,202501,1951.0,4073,M,6.0,4.0,1.0,32,320020.0,32004.0,1,2024-12-29,999990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,2.0,NaN,NaN,1.0,NaN,2025-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-29,NaN,NaN,NaN
3,2,A90,2024-12-29,202501,2024,32,320020,32004.0,2448025.0,2024-12-29,202501,2013.0,4011,F,5.0,4.0,3.0,32,320020.0,32004.0,1,2024-12-29,999992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,2.0,NaN,NaN,1.0,NaN,2025-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-29,NaN,NaN,NaN
4,2,A90,2024-12-29,202501,2024,32,320020,32004.0,2448025.0,2024-12-29,202501,1998.0,4026,M,6.0,1.0,5.0,32,320020.0,32004.0,1,2024-12-29,761240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,2.0,NaN,NaN,1.0,NaN,2025-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-29,NaN,NaN,NaN


## 5. Transform - Limpeza e Transformação

In [7]:
# =============================================================================
# TRANSFORM - ETAPA 1: SELEÇÃO DE COLUNAS
# =============================================================================

print('=' * 70)
print('ETAPA 2: TRANSFORM')
print('=' * 70)

print('\n[1/8] Selecionando colunas relevantes...')

colunas_existentes = [col for col in COLUNAS_PROCESSAMENTO if col in df_raw.columns]
colunas_faltantes = [col for col in COLUNAS_PROCESSAMENTO if col not in df_raw.columns]

if colunas_faltantes:
    print(f'   AVISO: Colunas não encontradas: {colunas_faltantes}')

df = df_raw[colunas_existentes].copy()

print(f'   Colunas selecionadas: {len(colunas_existentes)}')
print(f'   Colunas removidas: {len(df_raw.columns) - len(colunas_existentes)}')
print(f'   Shape atual: {df.shape}')

ETAPA 2: TRANSFORM

[1/8] Selecionando colunas relevantes...
   Colunas selecionadas: 37
   Colunas removidas: 84
   Shape atual: (1668787, 37)


In [8]:
# =============================================================================
# TRANSFORM - ETAPA 2: REMOÇÃO DE DUPLICATAS
# =============================================================================

print('\n[2/8] Removendo duplicatas...')

registros_inicial = len(df)
duplicatas_antes = df.duplicated().sum()
df = df.drop_duplicates()

print(f'   Duplicatas removidas: {duplicatas_antes:,}')
print(f'   Registros restantes: {len(df):,}')


[2/8] Removendo duplicatas...
   Duplicatas removidas: 7,153
   Registros restantes: 1,661,634


In [9]:
# =============================================================================
# TRANSFORM - ETAPA 3: CONVERSÃO DE DATAS
# =============================================================================

print('\n[3/8] Convertendo datas...')

# Converter colunas de data (formato YYYY-MM-DD do CSV)
colunas_data = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_OBITO']

for col in colunas_data:
    if col in df.columns:
        # Formato ISO: YYYY-MM-DD
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d', errors='coerce')

# Criar variáveis temporais derivadas
if 'DT_NOTIFIC' in df.columns:
    df['ano_notificacao'] = df['DT_NOTIFIC'].dt.year.astype('Int64')
    df['mes_notificacao'] = df['DT_NOTIFIC'].dt.month.astype('Int64')

if 'DT_SIN_PRI' in df.columns:
    df['semana_epi'] = df['DT_SIN_PRI'].dt.isocalendar().week.astype('Int64')

# Calcular intervalo entre sintomas e notificação
if 'DT_NOTIFIC' in df.columns and 'DT_SIN_PRI' in df.columns:
    df['dias_notificacao'] = (df['DT_NOTIFIC'] - df['DT_SIN_PRI']).dt.days

print(f'   Colunas de data convertidas: {len(colunas_data)}')
print(f'   Campos temporais criados: ano_notificacao, mes_notificacao, semana_epi, dias_notificacao')

# Exibir período dos dados (tratando NaT)
data_min = df['DT_NOTIFIC'].min()
data_max = df['DT_NOTIFIC'].max()
data_min_str = data_min.strftime('%d/%m/%Y') if pd.notna(data_min) else 'N/A'
data_max_str = data_max.strftime('%d/%m/%Y') if pd.notna(data_max) else 'N/A'
print(f"   Período dos dados: {data_min_str} a {data_max_str}")


[3/8] Convertendo datas...
   Colunas de data convertidas: 3
   Campos temporais criados: ano_notificacao, mes_notificacao, semana_epi, dias_notificacao
   Período dos dados: 29/12/2024 a 05/01/2026


In [10]:
# =============================================================================
# TRANSFORM - ETAPA 4: MAPEAMENTO DE CÓDIGOS SINAN
# =============================================================================

print('\n[4/8] Aplicando mapeamento de códigos SINAN...')

# Função auxiliar para mapeamento seguro
def map_safe(series, mapping, default='Nao informado'):
    return series.map(mapping).fillna(default)

# Aplicar mapeamentos
if 'SG_UF_NOT' in df.columns:
    df['uf_sigla'] = map_safe(df['SG_UF_NOT'], UF_MAP, 'Desconhecido')
    print('   uf_sigla criada')

if 'CS_SEXO' in df.columns:
    df['sexo_desc'] = map_safe(df['CS_SEXO'], SEXO_MAP)
    print('   sexo_desc criada')

if 'CS_RACA' in df.columns:
    df['raca_desc'] = map_safe(df['CS_RACA'], RACA_MAP)
    print('   raca_desc criada')

if 'CS_ESCOL_N' in df.columns:
    df['escolaridade_desc'] = map_safe(df['CS_ESCOL_N'], ESCOLARIDADE_MAP)
    print('   escolaridade_desc criada')

if 'CLASSI_FIN' in df.columns:
    df['classificacao_desc'] = map_safe(df['CLASSI_FIN'], CLASSIFICACAO_MAP, 'Em investigacao')
    print('   classificacao_desc criada')

if 'CRITERIO' in df.columns:
    df['criterio_desc'] = map_safe(df['CRITERIO'], CRITERIO_MAP)
    print('   criterio_desc criada')

if 'EVOLUCAO' in df.columns:
    df['evolucao_desc'] = map_safe(df['EVOLUCAO'], EVOLUCAO_MAP, 'Em investigacao')
    print('   evolucao_desc criada')

if 'CS_GESTANT' in df.columns:
    df['gestante_desc'] = map_safe(df['CS_GESTANT'], GESTANTE_MAP)
    print('   gestante_desc criada')

if 'TPAUTOCTO' in df.columns:
    df['autoctone_desc'] = map_safe(df['TPAUTOCTO'], AUTOCTONE_MAP)
    print('   autoctone_desc criada')

# Mapeamento de resultados laboratoriais
if 'RESUL_SORO' in df.columns:
    df['resultado_sorologia_desc'] = map_safe(df['RESUL_SORO'], RESUL_SORO_MAP)
    print('   resultado_sorologia_desc criada')

if 'RESUL_NS1' in df.columns:
    df['resultado_ns1_desc'] = map_safe(df['RESUL_NS1'], RESUL_NS1_MAP)
    print('   resultado_ns1_desc criada')

# Verificar distribuição da classificação
print('\n   Distribuição de Classificação Final:')
print(df['classificacao_desc'].value_counts())


[4/8] Aplicando mapeamento de códigos SINAN...
   uf_sigla criada
   sexo_desc criada
   raca_desc criada
   escolaridade_desc criada
   classificacao_desc criada
   criterio_desc criada
   evolucao_desc criada
   gestante_desc criada
   autoctone_desc criada
   resultado_sorologia_desc criada
   resultado_ns1_desc criada

   Distribuição de Classificação Final:
classificacao_desc
Dengue                         1408557
Inconclusivo                    170296
Em investigacao                  45573
Dengue com Sinais de Alarme      34572
Dengue Grave                      2636
Name: count, dtype: int64


In [11]:
# =============================================================================
# TRANSFORM - ETAPA 5: TRATAMENTO DE IDADE (Lógica SINAN)
# =============================================================================

print('\n[5/8] Tratando idade (lógica SINAN)...')

# Formato SINAN: TUUU onde T é o tipo e UUU é o valor
# T=1: Horas, T=2: Dias, T=3: Meses, T=4: Anos

def converter_idade_sinan(cod):
    if pd.isna(cod):
        return None
    cod = int(cod)
    tipo = cod // 1000
    valor = cod % 1000
    
    if tipo == 1:  # Horas
        return 0
    elif tipo == 2:  # Dias
        return 0
    elif tipo == 3:  # Meses
        return valor / 12
    elif tipo == 4:  # Anos
        return valor
    return None

if 'NU_IDADE_N' in df.columns:
    df['idade_anos'] = df['NU_IDADE_N'].apply(converter_idade_sinan)
    
    # Validar: idades fora do range 0-120 viram nulo
    df.loc[~df['idade_anos'].between(0, 120), 'idade_anos'] = None

# Criar faixas etárias
def categorizar_faixa_etaria(idade):
    if pd.isna(idade):
        return 'Nao informado'
    elif idade < 1:
        return '< 1 ano'
    elif idade < 5:
        return '1-4 anos'
    elif idade < 10:
        return '5-9 anos'
    elif idade < 20:
        return '10-19 anos'
    elif idade < 40:
        return '20-39 anos'
    elif idade < 60:
        return '40-59 anos'
    else:
        return '60+ anos'

df['faixa_etaria'] = df['idade_anos'].apply(categorizar_faixa_etaria)

print(f"   Idade convertida - Média: {df['idade_anos'].mean():.1f} anos")
print(f"   Idade convertida - Mediana: {df['idade_anos'].median():.0f} anos")
print('\n   Distribuição por faixa etária:')
print(df['faixa_etaria'].value_counts())


[5/8] Tratando idade (lógica SINAN)...
   Idade convertida - Média: 36.0 anos
   Idade convertida - Mediana: 34 anos

   Distribuição por faixa etária:
faixa_etaria
20-39 anos       588001
40-59 anos       449445
10-19 anos       238612
60+ anos         236944
5-9 anos          88638
1-4 anos          45258
< 1 ano           14690
Nao informado        46
Name: count, dtype: int64


In [12]:
# =============================================================================
# TRANSFORM - ETAPA 6: CRIAÇÃO DE AGREGADOS (Sintomas, Alarmes, Comorbidades)
# =============================================================================

print('\n[6/8] Criando campos agregados...')

# Contar sintomas presentes (1 = Sim)
SINTOMAS_COLS = ['FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO',
                 'NAUSEA', 'DOR_COSTAS', 'ARTRALGIA', 'DOR_RETRO']
sintomas_disp = [c for c in SINTOMAS_COLS if c in df.columns]

if sintomas_disp:
    df['qtd_sintomas'] = df[sintomas_disp].apply(
        lambda row: sum(1 for v in row if v == 1), axis=1
    )
    print(f"   qtd_sintomas criada (média: {df['qtd_sintomas'].mean():.1f})")
else:
    df['qtd_sintomas'] = 0

# AGREGAÇÃO DE ALARMES (8 colunas → 1)
ALARME_COLS = ['ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG',
               'ALRM_HEMAT', 'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT']
alarmes_disp = [c for c in ALARME_COLS if c in df.columns]

if alarmes_disp:
    df['qtd_alarmes'] = df[alarmes_disp].apply(
        lambda row: sum(1 for v in row if v == 1), axis=1
    )
    print(f"   qtd_alarmes criada (média: {df['qtd_alarmes'].mean():.2f})")
else:
    df['qtd_alarmes'] = 0

# AGREGAÇÃO DE COMORBIDADES (4 colunas → 1)
COMORBIDADE_COLS = ['DIABETES', 'HIPERTENSA', 'RENAL', 'HEPATOPAT']
comorb_disp = [c for c in COMORBIDADE_COLS if c in df.columns]

if comorb_disp:
    df['fl_comorbidade'] = df[comorb_disp].apply(
        lambda row: 1 if any(v == 1 for v in row) else 0, axis=1
    )
    print(f"   fl_comorbidade criada ({df['fl_comorbidade'].sum():,} pacientes com comorbidade)")
else:
    df['fl_comorbidade'] = 0


[6/8] Criando campos agregados...
   qtd_sintomas criada (média: 3.8)
   qtd_alarmes criada (média: 0.04)
   fl_comorbidade criada (197,224 pacientes com comorbidade)


In [13]:
# =============================================================================
# TRANSFORM - ETAPA 7: CRIAÇÃO DE FLAGS E INDICADORES
# =============================================================================

print('\n[7/8] Criando flags e indicadores epidemiológicos...')

# Flag: Caso confirmado de Dengue (10, 11, 12)
if 'CLASSI_FIN' in df.columns:
    df['fl_confirmado'] = np.where(df['CLASSI_FIN'].isin([10, 11, 12]), 1, 0)
    print(f"   fl_confirmado criada ({df['fl_confirmado'].sum():,} casos confirmados)")

# Flag: Caso grave (Dengue c/ Alarme ou Grave)
if 'CLASSI_FIN' in df.columns:
    df['fl_grave'] = np.where(df['CLASSI_FIN'].isin([11, 12]), 1, 0)
    print(f"   fl_grave criada ({df['fl_grave'].sum():,} casos graves)")

# Flag: Óbito
if 'EVOLUCAO' in df.columns:
    df['fl_obito'] = np.where(df['EVOLUCAO'] == 2, 1, 0)
    print(f"   fl_obito criada ({df['fl_obito'].sum():,} óbitos)")

# Flag: Hospitalizado
if 'HOSPITALIZ' in df.columns:
    df['fl_hospitalizado'] = np.where(df['HOSPITALIZ'] == 1, 1, 0)
    print(f"   fl_hospitalizado criada ({df['fl_hospitalizado'].sum():,} hospitalizações)")

# Calcular indicadores
total_confirmados = df['fl_confirmado'].sum()
if total_confirmados > 0:
    taxa_gravidade = (df['fl_grave'].sum() / total_confirmados) * 100
    taxa_letalidade = (df['fl_obito'].sum() / total_confirmados) * 100
    taxa_hospitalizacao = (df['fl_hospitalizado'].sum() / total_confirmados) * 100
    
    print(f'\n   INDICADORES EPIDEMIOLÓGICOS:')
    print(f'   Taxa de gravidade: {taxa_gravidade:.2f}%')
    print(f'   Taxa de letalidade: {taxa_letalidade:.3f}%')
    print(f'   Taxa de hospitalização: {taxa_hospitalizacao:.2f}%')


[7/8] Criando flags e indicadores epidemiológicos...
   fl_confirmado criada (1,445,765 casos confirmados)
   fl_grave criada (37,208 casos graves)
   fl_obito criada (1,773 óbitos)
   fl_hospitalizado criada (72,684 hospitalizações)

   INDICADORES EPIDEMIOLÓGICOS:
   Taxa de gravidade: 2.57%
   Taxa de letalidade: 0.123%
   Taxa de hospitalização: 5.03%


In [14]:
# =============================================================================
# TRANSFORM - ETAPA 8: TRATAMENTO DE VALORES INVÁLIDOS E SELEÇÃO FINAL
# =============================================================================

print('\n[8/8] Finalizando transformações...')

# Limpar intervalos de notificação absurdos (-7 a 365 dias é válido)
if 'dias_notificacao' in df.columns:
    invalidos = ~df['dias_notificacao'].between(-7, 365)
    df.loc[invalidos, 'dias_notificacao'] = None
    print(f'   dias_notificacao: {invalidos.sum():,} valores inválidos tratados')

# Seleção final de colunas para o esquema Silver
COLUNAS_SILVER = [
    # Localização
    'uf_sigla',
    
    # Tempo
    'DT_NOTIFIC', 'DT_SIN_PRI', 'DT_OBITO',
    'ano_notificacao', 'mes_notificacao', 'semana_epi', 'dias_notificacao',
    
    # Demografia
    'idade_anos', 'faixa_etaria', 'sexo_desc', 'raca_desc',
    'escolaridade_desc', 'gestante_desc',
    
    # Clínico
    'qtd_sintomas', 'qtd_alarmes', 'fl_comorbidade',
    'resultado_sorologia_desc', 'resultado_ns1_desc',
    
    # Desfecho
    'classificacao_desc', 'criterio_desc', 'evolucao_desc', 'autoctone_desc',
    
    # Flags
    'fl_confirmado', 'fl_grave', 'fl_obito', 'fl_hospitalizado'
]

# Selecionar apenas colunas existentes
colunas_finais = [col for col in COLUNAS_SILVER if col in df.columns]
df_silver = df[colunas_finais].copy()

# Renomear colunas para snake_case
rename_map = {
    'DT_NOTIFIC': 'data_notificacao',
    'DT_SIN_PRI': 'data_sintomas',
    'DT_OBITO': 'data_obito'
}

df_silver = df_silver.rename(columns=rename_map)

print(f'\n   Shape final: {df_silver.shape}')
print(f'   Colunas finais: {len(df_silver.columns)}')
print(f'   Colunas: {list(df_silver.columns)}')


[8/8] Finalizando transformações...
   dias_notificacao: 17 valores inválidos tratados

   Shape final: (1661634, 27)
   Colunas finais: 27
   Colunas: ['uf_sigla', 'data_notificacao', 'data_sintomas', 'data_obito', 'ano_notificacao', 'mes_notificacao', 'semana_epi', 'dias_notificacao', 'idade_anos', 'faixa_etaria', 'sexo_desc', 'raca_desc', 'escolaridade_desc', 'gestante_desc', 'qtd_sintomas', 'qtd_alarmes', 'fl_comorbidade', 'resultado_sorologia_desc', 'resultado_ns1_desc', 'classificacao_desc', 'criterio_desc', 'evolucao_desc', 'autoctone_desc', 'fl_confirmado', 'fl_grave', 'fl_obito', 'fl_hospitalizado']


In [15]:
# =============================================================================
# RESUMO DA TRANSFORMAÇÃO
# =============================================================================

print('\n' + '=' * 70)
print('RESUMO DA TRANSFORMAÇÃO')
print('=' * 70)

print(f'\nDADOS:')
print(f'   Registros originais: {len(df_raw):,}')
print(f'   Registros Silver: {len(df_silver):,}')
print(f'   Redução: {len(df_raw) - len(df_silver):,}')

print(f'\nCOLUNAS:')
print(f'   Originais: {len(df_raw.columns)}')
print(f'   Silver: {len(df_silver.columns)}')

print(f'\nINDICADORES:')
print(f"   Casos confirmados: {df_silver['fl_confirmado'].sum():,}")
print(f"   Casos graves: {df_silver['fl_grave'].sum():,}")
print(f"   Óbitos: {df_silver['fl_obito'].sum():,}")
print(f"   UFs: {df_silver['uf_sigla'].nunique()}")


RESUMO DA TRANSFORMAÇÃO

DADOS:
   Registros originais: 1,668,787
   Registros Silver: 1,661,634
   Redução: 7,153

COLUNAS:
   Originais: 121
   Silver: 27

INDICADORES:
   Casos confirmados: 1,445,765
   Casos graves: 37,208
   Óbitos: 1,773
   UFs: 27


## 6. Load - Salvamento dos Dados

In [ ]:
# =============================================================================
# LOAD - ETAPA 1: CRIAR SCHEMA E TABELA NO POSTGRESQL
# =============================================================================

print('=' * 70)
print('ETAPA 3: LOAD')
print('=' * 70)

print('\n[1/2] Criando schema e tabela no PostgreSQL...')

# Mapeamento de tipos de dados para cada coluna
COLUMN_TYPES = {
    # Localização
    'uf_sigla': 'TEXT NOT NULL',
    
    # Datas
    'data_notificacao': 'DATE',
    'data_sintomas': 'DATE',
    'data_obito': 'DATE',
    
    # Campos temporais derivados
    'ano_notificacao': 'INTEGER',
    'mes_notificacao': 'INTEGER',
    'semana_epi': 'INTEGER',
    'dias_notificacao': 'INTEGER',
    
    # Demografia
    'idade_anos': 'REAL',
    'faixa_etaria': 'TEXT NOT NULL',
    'sexo_desc': 'TEXT NOT NULL',
    'raca_desc': 'TEXT NOT NULL',
    'escolaridade_desc': 'TEXT NOT NULL',
    'gestante_desc': 'TEXT NOT NULL',
    
    # Clínico
    'qtd_sintomas': 'INTEGER NOT NULL',
    'qtd_alarmes': 'INTEGER NOT NULL',
    'fl_comorbidade': 'INTEGER NOT NULL',
    'resultado_sorologia_desc': 'TEXT NOT NULL',
    'resultado_ns1_desc': 'TEXT NOT NULL',
    
    # Classificação/Desfecho
    'classificacao_desc': 'TEXT NOT NULL',
    'criterio_desc': 'TEXT NOT NULL',
    'evolucao_desc': 'TEXT NOT NULL',
    'autoctone_desc': 'TEXT NOT NULL',
    
    # Flags
    'fl_confirmado': 'INTEGER NOT NULL',
    'fl_grave': 'INTEGER NOT NULL',
    'fl_obito': 'INTEGER NOT NULL',
    'fl_hospitalizado': 'INTEGER NOT NULL'
}

# Gerar DDL dinamicamente baseado nas colunas do DataFrame
def generate_ddl(df, column_types):
    columns_ddl = []
    # Adicionar chave primária auto-incremento
    columns_ddl.append('    id_notificacao SERIAL PRIMARY KEY')
    
    for col in df.columns:
        col_type = column_types.get(col, 'TEXT')
        # Escapar nomes de colunas em maiúsculo
        col_name = f'"{col}"' if col.isupper() else col
        columns_ddl.append(f'    {col_name} {col_type}')
    
    return ',\n'.join(columns_ddl)

DDL_TABLE = f'''
CREATE SCHEMA IF NOT EXISTS silver;

DROP TABLE IF EXISTS silver.dengue_silver CASCADE;

CREATE TABLE silver.dengue_silver (
{generate_ddl(df_silver, COLUMN_TYPES)}
);

CREATE INDEX idx_dengue_silver_uf ON silver.dengue_silver(uf_sigla);
CREATE INDEX idx_dengue_silver_ano ON silver.dengue_silver(ano_notificacao);
CREATE INDEX idx_dengue_silver_semana ON silver.dengue_silver(semana_epi);
CREATE INDEX idx_dengue_silver_classificacao ON silver.dengue_silver(classificacao_desc);
CREATE INDEX idx_dengue_silver_confirmado ON silver.dengue_silver(fl_confirmado);
CREATE INDEX idx_dengue_silver_data_notif ON silver.dengue_silver(data_notificacao);
CREATE INDEX idx_dengue_silver_data_sintomas ON silver.dengue_silver(data_sintomas);
'''

print(f'   Colunas na tabela: {len(df_silver.columns)}')

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute(DDL_TABLE)
    print("   Tabela 'silver.dengue_silver' criada")
    
    conn.commit()
    cur.close()
    conn.close()
    
except Exception as e:
    print(f'\n   AVISO: Não foi possível conectar ao banco: {e}')
    print('   Verifique se o Docker está rodando: docker-compose up -d')


ETAPA 3: LOAD

[1/2] Criando schema e tabela no PostgreSQL...
   Colunas na tabela: 27
   Tabela 'public.dengue_silver' criada


In [ ]:
# =============================================================================
# LOAD - ETAPA 2: INSERÇÃO DOS DADOS
# =============================================================================

print('\n[2/2] Inserindo dados no PostgreSQL...')

def prepare_value(val):
    """Prepara valor para inserção no banco"""
    if pd.isna(val):
        return None
    if isinstance(val, (np.integer, np.floating)):
        if np.isnan(val) or np.isinf(val):
            return None
        return float(val) if isinstance(val, np.floating) else int(val)
    if isinstance(val, pd.Timestamp):
        return val.date()
    return val

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    print('   Preparando dados para inserção...')
    
    # Colunas na ordem do DDL
    colunas_insert = list(df_silver.columns)
    
    data = []
    for _, row in df_silver.iterrows():
        record = tuple(prepare_value(row.get(col)) for col in colunas_insert)
        data.append(record)
    
    placeholders = ', '.join(['%s'] * len(colunas_insert))
    
    # Escapar nomes de colunas com aspas duplas para colunas em maiúsculo
    colunas_escaped = [f'"{col}"' if col.isupper() else col for col in colunas_insert]
    
    insert_sql = f'''
        INSERT INTO silver.dengue_silver ({', '.join(colunas_escaped)})
        VALUES ({placeholders})
    '''
    
    print(f'   Inserindo {len(data):,} registros...')
    execute_batch(cur, insert_sql, data, page_size=5000)
    
    conn.commit()
    
    cur.execute('SELECT COUNT(*) FROM silver.dengue_silver')
    count = cur.fetchone()[0]
    print(f'   Registros inseridos: {count:,}')
    
    cur.close()
    conn.close()
    
    print('\n   Dados carregados com sucesso no PostgreSQL!')
    
except Exception as e:
    print(f'\n   ERRO ao inserir dados: {e}')



[2/2] Inserindo dados no PostgreSQL...
   Preparando dados para inserção...
   Inserindo 1,661,634 registros...
   Registros inseridos: 1,661,634

   Dados carregados com sucesso no PostgreSQL!


In [ ]:
# =============================================================================
# VERIFICAÇÃO FINAL
# =============================================================================

print('\n' + '=' * 70)
print('VERIFICAÇÃO FINAL')
print('=' * 70)

try:
    conn = psycopg2.connect(**DB_CONFIG)
    
    print('\nAmostra dos dados no banco:')
    query_sample = '''
        SELECT uf_sigla, data_notificacao, data_sintomas, faixa_etaria, classificacao_desc, 
               fl_confirmado, fl_grave, fl_obito
        FROM silver.dengue_silver
        ORDER BY data_notificacao DESC
        LIMIT 10
    '''
    df_sample = pd.read_sql(query_sample, conn)
    display(df_sample)
    
    print('\nEstatísticas agregadas:')
    query_stats = '''
        SELECT 
            COUNT(*) as total_registros,
            COUNT(DISTINCT uf_sigla) as total_ufs,
            SUM(fl_confirmado) as casos_confirmados,
            SUM(fl_grave) as casos_graves,
            SUM(fl_obito) as obitos,
            SUM(fl_hospitalizado) as hospitalizacoes,
            MIN(data_notificacao) as data_inicio,
            MAX(data_notificacao) as data_fim
        FROM silver.dengue_silver
    '''
    df_stats = pd.read_sql(query_stats, conn)
    display(df_stats)
    
    print('\nTop 5 UFs por casos confirmados:')
    query_uf = '''
        SELECT uf_sigla, 
               SUM(fl_confirmado) as casos,
               SUM(fl_obito) as obitos,
               ROUND(SUM(fl_obito)::numeric / NULLIF(SUM(fl_confirmado), 0) * 100, 3) as letalidade_pct
        FROM silver.dengue_silver
        GROUP BY uf_sigla
        ORDER BY casos DESC
        LIMIT 5
    '''
    df_uf = pd.read_sql(query_uf, conn)
    display(df_uf)
    
    conn.close()
    
except Exception as e:
    print(f'Não foi possível verificar o banco: {e}')



VERIFICAÇÃO FINAL

Amostra dos dados no banco:


,uf_sigla,data_notificacao,data_sintomas,faixa_etaria,classificacao_desc,fl_confirmado,fl_grave,fl_obito
0,ES,2026-01-05,2025-12-20,20-39 anos,Em investigacao,0,0,0
1,ES,2026-01-05,2026-01-02,40-59 anos,Em investigacao,0,0,0
2,ES,2026-01-05,2025-12-30,20-39 anos,Em investigacao,0,0,0
3,ES,2026-01-05,2026-01-03,40-59 anos,Dengue,1,0,0
4,ES,2026-01-05,2026-01-03,20-39 anos,Em investigacao,0,0,0
5,ES,2026-01-05,2026-01-03,< 1 ano,Em investigacao,0,0,0
6,ES,2026-01-05,2026-01-03,40-59 anos,Em investigacao,0,0,0
7,ES,2026-01-05,2025-12-30,1-4 anos,Em investigacao,0,0,0
8,ES,2026-01-05,2026-01-02,20-39 anos,Em investigacao,0,0,0
9,ES,2026-01-05,2026-01-02,20-39 anos,Dengue,1,0,0



Estatísticas agregadas:


,total_registros,total_ufs,casos_confirmados,casos_graves,obitos,hospitalizacoes,data_inicio,data_fim
0,1661634,27,1445765,37208,1773,72684,2024-12-29,2026-01-05



Top 5 UFs por casos confirmados:


,uf_sigla,casos,obitos,letalidade_pct
0,SP,876832,1118,0.128
1,MG,119016,149,0.125
2,GO,96685,105,0.109
3,PR,92514,145,0.157
4,RS,44075,53,0.120


## 7. Conclusão

O processo ETL Raw → Silver foi concluído com sucesso!

**Transformações realizadas:**
- Remoção de colunas com >90% de valores nulos (identificadas na análise bronze)
- Conversão de datas do formato SINAN (DD/MM/YYYY)
- Decodificação de códigos SINAN para descrições legíveis (UF, sexo, raça, classificação, etc.)
- Tratamento de idade com lógica específica SINAN (TUUU)
- Agregação de sintomas, alarmes e comorbidades em contadores
- Criação de flags epidemiológicas (confirmado, grave, óbito, hospitalizado)
- Criação de campos temporais (ano, mês, semana epidemiológica)

**Destino dos dados:**
- **PostgreSQL:** `silver.dengue_silver`

